In [ ]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}
.code_block {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-size: 75%;
    line-height: 22px; /* 5px +12px + 5px */
    #text-indent: 25px;
    #background-color: #fbfbea;
    padding: 5px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Download and Installation
</center>
</h1>

In [ ]:
import requests
import os
import sys
import subprocess

<div class=h1_cell>
<p>
Download the spark tarball in the current directory. The URL is one of many mirrors listed on spark's official website.
</div>

In [6]:
spark_url = "http://apache.osuosl.org/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz"
r = requests.get(spark_url, stream=True)
filename = spark_url.rsplit('/')[-1]
with open(filename, 'wb') as f:
    f.write(r.content)

<div class=h1_cell>
<p>
Extract the spark tarball in the 'spark/' directory.
</div>

In [16]:
subprocess.call('mkdir spark'.split(' '))
subprocess.call('tar -xf spark-2.3.0-bin-hadoop2.7.tgz -C spark --strip-components 1'.split(' '))

0

<div class=h1_cell>
<p>
Add spark_home environment variable.
<p>
Add spark_home + '/bin' to run a pyspark console.
<p>
Add spark_home + '/python*' to environment to import pyspark.
<p>
NOTE: Run this chunk even if you already have Spark installed.
</div>

In [8]:
os.environ['SPARK_HOME'] = os.environ['HOME'] + '/spark'
os.environ['PATH'] += ':' + os.environ['SPARK_HOME'] + '/bin'
sys.path.append(os.environ['SPARK_HOME'] + '/python')
sys.path.append(os.environ['SPARK_HOME'] + '/python/lib/py4j-0.10.6-src.zip')

<div class=h1_cell>
<p>
Write the names of the slave nodes in the cluster. This script currently assumes the master machine is blue0.
</div>

In [9]:
machines = ["instance-2", "instance-3", "instance-4"]
content = "\n".join(machines)
with open(os.environ['SPARK_HOME'] + "/conf/slaves", 'w') as f:
    f.write(content)

<div class=h1_cell>
<p>
We now need to do the same exact thing on all the slave nodes. We will:
<ul>
<li>
Convert this notebook to a python file.
<li>
Delete the lines after the comment in the code below.
<li>
Run the editted python script on each slave node.
</ul>
<p>
Note: These nodes must have password-less ssh tunneling configured.
</div>

In [10]:
# Convert this ipython notebook to python script
!jupyter nbconvert --to=python setup_spark.ipynb

[NbConvertApp] Converting notebook setup_spark.ipynb to python
[NbConvertApp] Writing 5526 bytes to setup_spark.py


In [11]:
read_file = open('setup_spark.py', 'r')
lines = read_file.readlines()
read_file.close()

del lines[6]  # get_ipython is called for the style chunk, remove this

with open('setup_spark.py', 'w') as f:
    i = 0
    while i < len(lines) and lines[i].strip() != '# Convert this ipython notebook to python script':
        f.write(lines[i])
        i += 1

<div class=h1_cell>
<p>
This will take awhile. For each node, the script is downloading spark, extracting the package and configuring the environment.
</div>

In [15]:
!ssh instance-2 python < setup_spark.py
!ssh instance-3 python < setup_spark.py
!ssh instance-4 python < setup_spark.py

<h1>
<center>
Using PySpark
</center>
</h1>

<div class=h1_cell>
<p>
Start the cluster.
</div>

In [17]:
# run start-all.sh
subprocess.call(os.environ['SPARK_HOME'] + "/sbin/start-all.sh", env=os.environ)

0

<div class=h1_cell>
<p>
This is where most other programs regarding this project will start.
</div>

In [20]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession(SparkContext(master='spark://instance-1:7077'))
spark

<div class=h1_cell>
<p>
Lets generate some semi-random data and run a Spark K-Means implementation on it. We'll create a 2D array with 4 centers.
</div>

In [21]:
# 4 clusters
from random import random, shuffle

upper_left = [[random()*0.5, random()*0.5 + 0.5] for _ in range(2500)]
upper_right = [[random()*0.5 + 0.5 for _ in range(2)] for _ in range(2500)]
bottom_left = [[random()*0.5 for _ in range(2)] for _ in range(2500)]
bottom_right = [[random()*.5 + 0.5, random()*0.5] for _ in range(2500)]

matrix = upper_left + upper_right + bottom_left + bottom_right
shuffle(matrix)

data = spark.createDataFrame(matrix, schema=["A", "B"])
data.show(5)

+--------------------+--------------------+
|                   A|                   B|
+--------------------+--------------------+
| 0.09264110029539968| 0.19437044681165994|
|0.012536162787798177|  0.2739293070646507|
|  0.4902528211318116|0.006829614908638426|
| 0.01572251213870518|  0.5741359312992989|
|  0.6339949288333049| 0.32263475086230514|
+--------------------+--------------------+
only showing top 5 rows



In [24]:
from pyspark.ml.feature import VectorAssembler

vdf = VectorAssembler(inputCols=data.columns, outputCol="features").transform(data)
vdf.show(5)

+--------------------+--------------------+--------------------+
|                   A|                   B|            features|
+--------------------+--------------------+--------------------+
| 0.09264110029539968| 0.19437044681165994|[0.09264110029539...|
|0.012536162787798177|  0.2739293070646507|[0.01253616278779...|
|  0.4902528211318116|0.006829614908638426|[0.49025282113181...|
| 0.01572251213870518|  0.5741359312992989|[0.01572251213870...|
|  0.6339949288333049| 0.32263475086230514|[0.63399492883330...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [25]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=4, maxIter=10, initMode="random")
model = kmeans.fit(vdf)

wssse = model.computeCost(vdf)
print("Within Set Sum of Squared Errors = " + str(wssse))

print("Centers:")
model.clusterCenters()

Within Set Sum of Squared Errors = 416.921486518
Centers:


[array([0.25413505, 0.25934256]),
 array([0.25156688, 0.75979991]),
 array([0.74504744, 0.24207099]),
 array([0.75325381, 0.74859811])]

<div class=h1_cell>
<p>
You can compare this to a python-only single-machine k-means to get an idea of performance gain.
</div>

In [19]:
spark.stop()

In [ ]:
subprocess.call(os.environ['SPARK_HOME'] + "/sbin/stop-all.sh")